# アンサンブルに使うモデルたち

## Xgboost 過学習

In [30]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv("test.csv")
train_df_y = train_df[["y"]]
train_df = train_df.drop(["y"],axis=1)
train_df_y.reset_index()
df = pd.concat([train_df,test_df])
df['datetime'] = pd.to_datetime(df.datetime)
#何月か
df['Month'] = df['datetime'].dt.month
#何日か
df['DayofMonth'] = df['datetime'].dt.day
#何曜日か
df['dayofweek'] = df['datetime'].dt.dayofweek
#何年か
df['year'] = df['datetime'].dt.year
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]

In [49]:
df_h1

,datetime,client,close,price_am,price_pm,Month,DayofMonth,dayofweek,year,y
0,2010-07-01,0,0,-1,-1,7,1,3,2010,17
1,2010-07-02,0,0,-1,-1,7,2,4,2010,18
2,2010-07-03,0,0,-1,-1,7,3,5,2010,20
3,2010-07-04,0,0,-1,-1,7,4,6,2010,20
4,2010-07-05,0,0,-1,-1,7,5,0,2010,14
5,2010-07-06,0,0,-1,-1,7,6,1,2010,14
6,2010-07-07,0,0,-1,-1,7,7,2,2010,4
7,2010-07-08,0,0,-1,-1,7,8,3,2010,10
8,2010-07-09,0,0,-1,-1,7,9,4,2010,12
9,2010-07-10,0,0,-1,-1,7,10,5,2010,21


In [52]:
#2010-07-01から2016-03-31
#2016-04-01から2017-03-31
#2015-03-25から2016-03-31
df_h = train_df[train_df["datetime"]>="2015-03-30"]
df_h1 = train_df
#xgboostで回帰モデル生成
import xgboost as xgb
#モデルのインスタンス作成
#5 300
mod = xgb.XGBRegressor(max_depth =30,n_estimators=500)
X = df_h1.drop(["datetime","y","year"], axis=1)
 

Y = df_h1["y"]
#欠損値のデータを学習させずにモデル生成
mod.fit(X,Y,eval_metric=['mae'])

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=30,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [53]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(Y,mod.predict(X))

0.0011060675486447072

In [57]:
df_h1["mod_predict"] = mod.predict(X)

In [77]:
df_h1.to_csv("mod_predict_train.csv", index=False)

In [78]:
df_h1

,datetime,client,close,price_am,price_pm,Month,DayofMonth,dayofweek,year,y,mod_predict
0,2010-07-01,0,0,-1,-1,7,1,3,2010,17,16.998917
1,2010-07-02,0,0,-1,-1,7,2,4,2010,18,17.999874
2,2010-07-03,0,0,-1,-1,7,3,5,2010,20,19.999239
3,2010-07-04,0,0,-1,-1,7,4,6,2010,20,20.000753
4,2010-07-05,0,0,-1,-1,7,5,0,2010,14,14.000554
5,2010-07-06,0,0,-1,-1,7,6,1,2010,14,13.999285
6,2010-07-07,0,0,-1,-1,7,7,2,2010,4,4.001387
7,2010-07-08,0,0,-1,-1,7,8,3,2010,10,9.999783
8,2010-07-09,0,0,-1,-1,7,9,4,2010,12,12.000208
9,2010-07-10,0,0,-1,-1,7,10,5,2010,21,20.998043


In [18]:
xgb_pred = mod.predict(test_df.drop(["datetime","year"], axis=1))
test_df["y"] = xgb_pred 
test_df["y"][test_df["close"]==1] = int(0)
submission = pd.concat([test_df.datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/haradakeno/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.htm

In [14]:
xgb_pred = mod.predict(test_df.drop(["datetime","year"], axis=1))
test_df["y"] = xgb_pred + 6.3793867979
test_df["y"][test_df["close"]==1] = int(0)
submission = pd.concat([test_df.datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/haradakeno/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.htm

## ニューラルネット

In [69]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization

In [70]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv("test.csv")
train_df_y = train_df[["y"]]
train_df = train_df.drop(["y"],axis=1)
train_df_y.reset_index()
df = pd.concat([train_df,test_df])
df['datetime'] = pd.to_datetime(df.datetime)
#何月か
df['Month'] = df['datetime'].dt.month
#何日か
df['DayofMonth'] = df['datetime'].dt.day
#何曜日か
df['dayofweek'] = df['datetime'].dt.dayofweek
#何年か
df['year'] = df['datetime'].dt.year
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df["year"])

df["year_num"] = le.transform(df["year"])
#ダミー変数化
#何月か
month_df = pd.get_dummies(df['Month'], prefix='M', prefix_sep='_')
#何日か
dayofmonth_df = pd.get_dummies(df["DayofMonth"], prefix='DM', prefix_sep='_')
#何曜日か
dayofweek_df = pd.get_dummies(df['dayofweek'], prefix='DW', prefix_sep='_')
#何年か
year_df= pd.get_dummies(df['year'], prefix='y', prefix_sep='_')
#priceはどうか 
price_am_df = pd.get_dummies(df['price_am'], prefix='PA', prefix_sep='_')
price_pm_df = pd.get_dummies(df['price_pm'], prefix='PP', prefix_sep='_')
#datetimeを保存
df_id = df[['datetime']]
#year_df削除
df = pd.concat([df,
                month_df,
                dayofmonth_df,
                dayofweek_df,
               price_am_df,
               price_pm_df,
               year_df], axis=1)
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]
test_df_datetime = test_df["datetime"]
test_df_y_2017= test_df["y_2017"]

df = df.drop(['datetime','price_am', 'price_pm', 'Month',
       'DayofMonth', 'dayofweek',"year_num","year",'M_12','DM_31','DW_6','PA_5','PP_5','y_2017'],axis=1)
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]

In [71]:
df.columns

Index(['client', 'close', 'M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6', 'M_7',
       'M_8', 'M_9', 'M_10', 'M_11', 'DM_1', 'DM_2', 'DM_3', 'DM_4', 'DM_5',
       'DM_6', 'DM_7', 'DM_8', 'DM_9', 'DM_10', 'DM_11', 'DM_12', 'DM_13',
       'DM_14', 'DM_15', 'DM_16', 'DM_17', 'DM_18', 'DM_19', 'DM_20', 'DM_21',
       'DM_22', 'DM_23', 'DM_24', 'DM_25', 'DM_26', 'DM_27', 'DM_28', 'DM_29',
       'DM_30', 'DW_0', 'DW_1', 'DW_2', 'DW_3', 'DW_4', 'DW_5', 'PA_-1',
       'PA_0', 'PA_1', 'PA_2', 'PA_3', 'PA_4', 'PP_-1', 'PP_0', 'PP_1', 'PP_2',
       'PP_3', 'PP_4', 'y_2010', 'y_2011', 'y_2012', 'y_2013', 'y_2014',
       'y_2015', 'y_2016'],
      dtype='object')

In [29]:
model_1 = Sequential()
model_1.add(Dense(204,input_dim=68,activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.3))
model_1.add(Dense(102,activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.3))
model_1.add(Dense(51,activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.3))
model_1.add(Dense(37,activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.3))
model_1.add(Dense(1))
opt = keras.optimizers.Adam(lr = 0.06,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0003)
model_1.compile(optimizer=opt,
              loss = "mean_absolute_error",
              metrics=["mae"])
model_1.fit(x.values,y.values,nb_epoch=600,batch_size=300)

Epoch 1/600
2074/2074 [==============================] - 2s - loss: 32.6722 - mean_absolute_error: 32.6722     
Epoch 2/600
2074/2074 [==============================] - 0s - loss: 21.5103 - mean_absolute_error: 21.5103     
Epoch 3/600
2074/2074 [==============================] - 0s - loss: 10.9637 - mean_absolute_error: 10.9637     
Epoch 4/600
2074/2074 [==============================] - 0s - loss: 9.3198 - mean_absolute_error: 9.3198      
Epoch 5/600
2074/2074 [==============================] - 0s - loss: 8.4284 - mean_absolute_error: 8.4284     
Epoch 6/600
2074/2074 [==============================] - 0s - loss: 7.4853 - mean_absolute_error: 7.4853     
Epoch 7/600
2074/2074 [==============================] - 0s - loss: 7.1738 - mean_absolute_error: 7.1738     
Epoch 8/600
2074/2074 [==============================] - 0s - loss: 6.7272 - mean_absolute_error: 6.7272     
Epoch 9/600
2074/2074 [==============================] - 0s - loss: 6.6533 - mean_absolute_error: 6.6533     
Epo

2074/2074 [==============================] - 0s - loss: 4.7416 - mean_absolute_error: 4.7416     
Epoch 75/600
2074/2074 [==============================] - 0s - loss: 4.6205 - mean_absolute_error: 4.6205     
Epoch 76/600
2074/2074 [==============================] - 0s - loss: 4.8306 - mean_absolute_error: 4.8306     
Epoch 77/600
2074/2074 [==============================] - 0s - loss: 4.6938 - mean_absolute_error: 4.6938     
Epoch 78/600
2074/2074 [==============================] - 0s - loss: 4.7785 - mean_absolute_error: 4.7785     
Epoch 79/600
2074/2074 [==============================] - 0s - loss: 4.6108 - mean_absolute_error: 4.6108     
Epoch 80/600
2074/2074 [==============================] - 0s - loss: 4.5230 - mean_absolute_error: 4.5230     
Epoch 81/600
2074/2074 [==============================] - 0s - loss: 4.5591 - mean_absolute_error: 4.5591     
Epoch 82/600
2074/2074 [==============================] - 0s - loss: 4.5653 - mean_absolute_error: 4.5653     
Epoch 83/600
2

2074/2074 [==============================] - 0s - loss: 3.9246 - mean_absolute_error: 3.9246     
Epoch 148/600
2074/2074 [==============================] - 0s - loss: 4.0421 - mean_absolute_error: 4.0421     
Epoch 149/600
2074/2074 [==============================] - 0s - loss: 3.8648 - mean_absolute_error: 3.8648     
Epoch 150/600
2074/2074 [==============================] - 0s - loss: 3.9241 - mean_absolute_error: 3.9241     
Epoch 151/600
2074/2074 [==============================] - 0s - loss: 3.9969 - mean_absolute_error: 3.9969     
Epoch 152/600
2074/2074 [==============================] - 0s - loss: 3.9264 - mean_absolute_error: 3.9264     
Epoch 153/600
2074/2074 [==============================] - 0s - loss: 4.0960 - mean_absolute_error: 4.0960     
Epoch 154/600
2074/2074 [==============================] - 0s - loss: 3.9052 - mean_absolute_error: 3.9052     
Epoch 155/600
2074/2074 [==============================] - 0s - loss: 4.0738 - mean_absolute_error: 4.0738     
Epoch 

2074/2074 [==============================] - 0s - loss: 3.7235 - mean_absolute_error: 3.7235     
Epoch 221/600
2074/2074 [==============================] - 0s - loss: 3.8674 - mean_absolute_error: 3.8674     
Epoch 222/600
2074/2074 [==============================] - 0s - loss: 3.5406 - mean_absolute_error: 3.5406     
Epoch 223/600
2074/2074 [==============================] - 0s - loss: 3.6690 - mean_absolute_error: 3.6690     
Epoch 224/600
2074/2074 [==============================] - 0s - loss: 3.7155 - mean_absolute_error: 3.7155     
Epoch 225/600
2074/2074 [==============================] - 0s - loss: 3.5684 - mean_absolute_error: 3.5684     
Epoch 226/600
2074/2074 [==============================] - 0s - loss: 3.5626 - mean_absolute_error: 3.5626     
Epoch 227/600
2074/2074 [==============================] - 0s - loss: 3.6336 - mean_absolute_error: 3.6336     
Epoch 228/600
2074/2074 [==============================] - 0s - loss: 3.5348 - mean_absolute_error: 3.5348     
Epoch 

2074/2074 [==============================] - 0s - loss: 3.2203 - mean_absolute_error: 3.2203     
Epoch 294/600
2074/2074 [==============================] - 0s - loss: 3.4052 - mean_absolute_error: 3.4052     
Epoch 295/600
2074/2074 [==============================] - 0s - loss: 3.1829 - mean_absolute_error: 3.1829     
Epoch 296/600
2074/2074 [==============================] - 0s - loss: 3.4124 - mean_absolute_error: 3.4124     
Epoch 297/600
2074/2074 [==============================] - 0s - loss: 3.3811 - mean_absolute_error: 3.3811     
Epoch 298/600
2074/2074 [==============================] - 0s - loss: 3.1271 - mean_absolute_error: 3.1271     
Epoch 299/600
2074/2074 [==============================] - 0s - loss: 3.3580 - mean_absolute_error: 3.3580     
Epoch 300/600
2074/2074 [==============================] - 0s - loss: 3.3470 - mean_absolute_error: 3.3470     
Epoch 301/600
2074/2074 [==============================] - 0s - loss: 3.3763 - mean_absolute_error: 3.3763     
Epoch 

2074/2074 [==============================] - 0s - loss: 3.1254 - mean_absolute_error: 3.1254     
Epoch 367/600
2074/2074 [==============================] - 0s - loss: 3.1433 - mean_absolute_error: 3.1433     
Epoch 368/600
2074/2074 [==============================] - 0s - loss: 3.1631 - mean_absolute_error: 3.1631     
Epoch 369/600
2074/2074 [==============================] - 0s - loss: 3.0962 - mean_absolute_error: 3.0962     
Epoch 370/600
2074/2074 [==============================] - 0s - loss: 3.1235 - mean_absolute_error: 3.1235     
Epoch 371/600
2074/2074 [==============================] - 0s - loss: 2.9974 - mean_absolute_error: 2.9974     - ETA: 0s - loss: 3.0714 - mean_absolute_error: 3.07
Epoch 372/600
2074/2074 [==============================] - 0s - loss: 3.1996 - mean_absolute_error: 3.1996     
Epoch 373/600
2074/2074 [==============================] - 0s - loss: 3.2117 - mean_absolute_error: 3.2117     
Epoch 374/600
2074/2074 [==============================] - 0s - lo

2074/2074 [==============================] - 0s - loss: 2.9619 - mean_absolute_error: 2.9619     
Epoch 439/600
2074/2074 [==============================] - 0s - loss: 3.0272 - mean_absolute_error: 3.0272     
Epoch 440/600
2074/2074 [==============================] - 0s - loss: 2.8620 - mean_absolute_error: 2.8620     
Epoch 441/600
2074/2074 [==============================] - 0s - loss: 2.7659 - mean_absolute_error: 2.7659     
Epoch 442/600
2074/2074 [==============================] - 0s - loss: 2.9998 - mean_absolute_error: 2.9998     
Epoch 443/600
2074/2074 [==============================] - 0s - loss: 2.8998 - mean_absolute_error: 2.8998     
Epoch 444/600
2074/2074 [==============================] - 0s - loss: 3.0448 - mean_absolute_error: 3.0448     
Epoch 445/600
2074/2074 [==============================] - 0s - loss: 2.8627 - mean_absolute_error: 2.8627     
Epoch 446/600
2074/2074 [==============================] - 0s - loss: 2.8816 - mean_absolute_error: 2.8816     
Epoch 

2074/2074 [==============================] - 0s - loss: 2.9764 - mean_absolute_error: 2.9764     
Epoch 512/600
2074/2074 [==============================] - 0s - loss: 2.9993 - mean_absolute_error: 2.9993     
Epoch 513/600
2074/2074 [==============================] - 0s - loss: 2.8840 - mean_absolute_error: 2.8840     
Epoch 514/600
2074/2074 [==============================] - 0s - loss: 2.9711 - mean_absolute_error: 2.9711     
Epoch 515/600
2074/2074 [==============================] - 0s - loss: 2.8903 - mean_absolute_error: 2.8903     
Epoch 516/600
2074/2074 [==============================] - 0s - loss: 3.2762 - mean_absolute_error: 3.2762     
Epoch 517/600
2074/2074 [==============================] - 0s - loss: 2.8362 - mean_absolute_error: 2.8362     
Epoch 518/600
2074/2074 [==============================] - 0s - loss: 2.8048 - mean_absolute_error: 2.8048     
Epoch 519/600
2074/2074 [==============================] - 0s - loss: 2.8354 - mean_absolute_error: 2.8354     
Epoch 

2074/2074 [==============================] - 0s - loss: 2.7186 - mean_absolute_error: 2.7186     
Epoch 585/600
2074/2074 [==============================] - 0s - loss: 2.8555 - mean_absolute_error: 2.8555     
Epoch 586/600
2074/2074 [==============================] - 0s - loss: 2.8050 - mean_absolute_error: 2.8050     
Epoch 587/600
2074/2074 [==============================] - 0s - loss: 2.6693 - mean_absolute_error: 2.6693     
Epoch 588/600
2074/2074 [==============================] - 0s - loss: 2.7992 - mean_absolute_error: 2.7992     
Epoch 589/600
2074/2074 [==============================] - 0s - loss: 2.6311 - mean_absolute_error: 2.6311     
Epoch 590/600
2074/2074 [==============================] - 0s - loss: 2.6831 - mean_absolute_error: 2.6831     
Epoch 591/600
2074/2074 [==============================] - 0s - loss: 2.8676 - mean_absolute_error: 2.8676     
Epoch 592/600
2074/2074 [==============================] - 0s - loss: 2.6646 - mean_absolute_error: 2.6646     
Epoch 

In [25]:
train_df = train_df[train_df["close"]==0]
x = train_df.drop(["y"],axis=1)
y = train_df["y"]

In [64]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y,model_1.predict(x.values))

1.0022961796766996

In [72]:
train_df["neural_y"] = model_1.predict(train_df.drop(["y"],axis=1).values)
train_df["neural_y"] = (1 - train_df['close']) * train_df.neural_y


In [75]:
train_df[train_df["close"]==1]

,client,close,M_1,M_2,M_3,M_4,M_5,M_6,M_7,M_8,...,PP_4,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y,neural_y
183,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0.0
184,0,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
185,0,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
186,0,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
548,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
549,0,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0.0
550,0,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0.0
551,0,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0.0
914,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0.0
915,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0


In [74]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(train_df.y,train_df.neural_y)

0.98941564809589488

In [76]:
train_df.to_csv("neural_predict_train.csv",index=False)

In [83]:
#test_df["y_y"] = model_1.predict(test_df.values)
test_df["y_y"] = (1 - test_df['close']) * test_df.y_y 

submission = pd.concat([test_df_datetime, test_df.y_y], axis=1)
submission.to_csv("submission_neural_2.csv", index=False,header=False)
#14.5899958763

/Users/haradakeno/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [82]:
test_df

,client,close,M_1,M_2,M_3,M_4,M_5,M_6,M_7,M_8,...,PP_3,PP_4,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_y
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,72.324989
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,53.730869
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,71.987434
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,61.873383
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,55.293430
5,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,59.398621
6,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,64.232567
7,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,54.787140
8,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,68.312988
9,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,68.278282


In [ ]:
test_df["y_y"] = (1 - test_df['close']) * test_df.y_y 

submission = pd.concat([test_df_datetime, test_df.y_y], axis=1)
submission.to_csv("submission_neural_2.csv", index=False,header=False)

In [86]:
test_df = pd.concat([test_df,test_df_y_2017],axis=1)
test_df["y_y"] = (1 - test_df['close']) * test_df.y_y + (1 - test_df['close'])*(test_df["y_2017"])*8 
submission = pd.concat([test_df_datetime, test_df.y_y], axis=1)
submission.to_csv("submission_neural_3.csv", index=False,header=False)
test_df["y_y"][test_df["y_2017"]==1].mean()

TypeError: '<' not supported between instances of 'str' and 'int'

In [85]:
test_df

,client,close,M_1,M_2,M_3,M_4,M_5,M_6,M_7,M_8,...,PP_4,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_y,y_2017
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,72.324989,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,53.730869,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,71.987434,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,61.873383,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,55.293430,0
5,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,59.398621,0
6,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,64.232567,0
7,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,54.787140,0
8,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,68.312988,0
9,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,68.278282,0
